In [1]:
import os
import glob
import numpy as np
import cv2
import matplotlib.image as mpimg       
import matplotlib.pyplot as plt        
import matplotlib.patches as mpatches
from scipy.signal import argrelextrema
from skimage import measure  
import PIL
import pandas as pd
import scipy.ndimage as ndi
from scipy.signal import argrelextrema
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops
from skimage.filters import gabor

%matplotlib inline

In [2]:
classes = ['leaf1','leaf2','leaf3','leaf4','leaf5','leaf6','leaf1','leaf7','leaf8','leaf9','leaf10',
           'leaf11','leaf12','leaf13','leaf14','leaf15']

In [3]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return [rho, phi]

In [4]:
feats = pd.DataFrame([], columns = ['index','total_max','r','g','b','r_d','g_d','b_d','peri','x','y','w','h','asp_ratio',
                                    'rect','circ','lbp_energy','lbp_entropy','contrast','dissimilarity','homogeneity',
                                    'energy','correlation','gabor_energy','gabor_entropy'])
for i in range(0, len(classes)):
    print(classes[i])
    impth = os.path.join('datasetcrt/', classes[i], '*g')
    #print ( impth)
    c = 0
    for img_p in glob.glob(impth):
        print ( " {} /76".format(c) , end = '\r')
        c = c + 1
        img = mpimg.imread(img_p)
        #print ( img_p )
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gs, (25,25),0)
        thrsh_ot,img_thre = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        closing = cv2.morphologyEx(img_thre, cv2.MORPH_CLOSE, np.ones((50,50),np.uint8))
        
        image, contours, _ = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        cy, cx = ndi.center_of_mass(image)
        contours = measure.find_contours(image, .8)
        contour = max(contours, key=len)
        
        polar_contour = np.array([cart2pol(x, y) for x, y in contour])
        
        #shift to centre
        contour[::,1] -= cx  
        contour[::,0] -= cy
        
        
        # fro local maxima
        polar_contour = np.array([cart2pol(x, y) for x, y in contour])
        
        c_max_index = argrelextrema(polar_contour[::,0], np.greater, order=8)
        # for local minima
        c_min_index = argrelextrema(polar_contour[::,0], np.less, order=8)
        
        total_max = len( c_max_index[0] )
        total_min = len( c_min_index[0] )
        
        pix = img.reshape((len(img)), len(img[0]), 3)
        r,g,b = 0,0,0

        r = np.mean(pix[:][:][0])
        g = np.mean(pix[:][:][1])
        b = np.mean(pix[:][:][2])
        r_d = np.std(pix[:][:][0])
        g_d = np.std(pix[:][:][1])
        b_d = np.std(pix[:][:][2])
        
        peri = cv2.arcLength(cnt,True)
        #print ( peri )
        
        epsilon = 0.1*cv2.arcLength(cnt,True)
        approx = cv2.approxPolyDP(cnt,epsilon,True)
        hull = cv2.convexHull(cnt)
        k = cv2.isContourConvex(cnt)
        
        
        M = cv2.moments(cnt)
        area = cv2.contourArea(cnt)
        peri = cv2.arcLength(cnt,True)
        x,y,w,h = cv2.boundingRect(cnt)
        asp_ratio = float(w)/h
        if int(area) == 0:
            area = 1
        rect = w*h/area
        circ = ((peri)**2)/area
        
        

        
        img_arr = np.array(gs)
        
        
        # LBP
        feat_lbp = local_binary_pattern(img_arr,5,2,'uniform').reshape(len(img)*len(img[0]))
        lbp_hist,_ = np.histogram(feat_lbp,8)
        lbp_hist = np.array(lbp_hist,dtype=float)
        lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy = np.nansum(lbp_prob**2)
        lbp_entropy = -np.nansum(np.multiply(lbp_prob,np.log2(lbp_prob)))   
        
        
        # GLCM
        gCoMat = greycomatrix(img_arr, [2], [0],256,symmetric=True, normed=True)
        contrast = greycoprops(gCoMat, prop='contrast')
        dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
        homogeneity = greycoprops(gCoMat, prop='homogeneity')    
        energy = greycoprops(gCoMat, prop='energy')
        correlation = greycoprops(gCoMat, prop='correlation')    
        feat_glcm = np.array([contrast[0][0],dissimilarity[0][0],homogeneity[0][0],energy[0][0],correlation[0][0]])
        
        
        # Gabor filter
        gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
        gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
        gabor_hist,_ = np.histogram(gaborFilt,8)
        gabor_hist = np.array(gabor_hist,dtype=float)
        gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
        gabor_energy = np.nansum(gabor_prob**2)
        gabor_entropy = -np.nansum(np.multiply(gabor_prob,np.log2(gabor_prob)))
          
        
        feat = pd.DataFrame([[i,total_max,r,g,b,r_d,g_d,b_d,peri,x,y,w,h,asp_ratio,rect,circ,
                              lbp_energy,lbp_entropy,contrast[0][0],dissimilarity[0][0],homogeneity[0][0],
                              energy[0][0],correlation[0][0],gabor_energy,gabor_entropy]], columns = ['index','total_max','r','g','b','r_d','g_d','b_d','peri','x','y','w','h','asp_ratio',
                                    'rect','circ','lbp_energy','lbp_entropy','contrast','dissimilarity','homogeneity',
                                    'energy','correlation','gabor_energy','gabor_entropy'])
        feats = feats.append(feat)

leaf1


C:\Users\Abhi\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: divide by zero encountered in log2
C:\Users\Abhi\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in multiply


leaf276
leaf376
leaf476
leaf576
leaf676
leaf176
leaf776
leaf876
leaf976
leaf106
leaf116
leaf126
leaf136
leaf146
leaf156


In [5]:
feats.to_csv("feats.csv")